# Augmenting synthetic digits  
This notebook is used to create new, augmented pictures based on synthetically created pictures of digits. 

In [0]:
# mounting on Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
# load all necessary packages

from imgaug import augmenters as iaa
from matplotlib.pyplot import imshow
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob

%matplotlib inline

**Defining augmentation sequences and method to generate augmented images**

In [0]:
# function for drawing random lines on images

def img_func(images, random_state, parents, hooks):
    k = np.random.randint(1,59)

    for img in images:
        img[k] = 255
        img[k+1] = 255

    return images

def keypoint_func(keypoints_on_images, random_state, parents, hooks):
    return keypoints_on_images

In [0]:
# defining augmentation sequences

seq = iaa.Sequential([
    iaa.Lambda(img_func, keypoint_func),
    iaa.GaussianBlur(sigma=(1.0, 8.0)), 
    iaa.AdditiveGaussianNoise(scale=0.1*255),
    iaa.CoarseDropout(p=(0.03, 0.05), size_percent=0.1)
])

seq2 = iaa.Sequential([
    iaa.Affine(rotate=(-15,15),
               shear=(-10,10))
])

In [0]:
def augment_digits(files, output_dir, n_augmentations, nondigits=False):
  
  for idx, file in enumerate(files):
    # loading image
    img = Image.open(file, 'r')
    img = cv2.bitwise_not(np.asarray(img))

    try:
      for t in range(1, n_augmentations):
        
        # augmenting image
        augimg = seq(images=np.asarray(img))
        gray = cv2.cvtColor(augimg, cv2.COLOR_BGR2GRAY)
        auggray = seq2(images=gray)

        label = file.split(".")[-2][-1]

        # write augmented image to Drive 
        if nondigits:
          new_file_name = str(idx) + str(t) + "_n.jpg"
        else:
          new_file_name = file.split("/")[-1].replace(".", "_augmented_{}.".format(t)) 
          
        cv2.imwrite(output_dir + new_file_name, auggray)

    except:
      pass

**Generating augmented digits and nondigits**

In [0]:
# digit_input = glob.glob("/content/drive/My Drive/Team Bergere/Optimizer/Data/Synthetic_dataset/*.jpg")
# digit_output = "/content/drive/My Drive/Team Bergere/Optimizer/Data/augmented2/"

# augment_digits(digit_input, digit_output, n_augmentations=20, nondigits=False)

# nondigit_input = glob.glob("/content/drive/My Drive/Team Bergere/Optimizer/Data/optimizer/HQ_analog/Split/Train/10/*.jpg") # non-digits (error in cropping)
# nondigit_output = "/content/drive/My Drive/Team Bergere/Optimizer/Data/augmented3/"

# augment_digits(nondigit_input, nondigit_output, n_augmentations=200, nondigits=True)